In [1]:
import pandas as pd
import os
from pandasql import sqldf
import ast
import numpy as np

In [2]:
base_dir = r'C:\Users\adeyehat\Desktop\scrappy\homefund\property-pro'

In [3]:
dt = pd.read_csv(os.path.join(base_dir,r'data\nigerianpropertysale.csv'))

In [4]:
dt.rename(columns = {'description':'description_1'},inplace =True)

#### Main Details

In [5]:
dt['main_details'] = dt['main_details'].apply(lambda x:ast.literal_eval(x))

In [6]:
dt = pd.concat([dt,dt['main_details'].apply(pd.Series)],axis=1).drop(['main_details','depth','download_timeout','download_slot','download_latency','redirect_times','redirect_ttl','redirect_urls','redirect_reasons'],axis = 1)

#### Agent Details

In [7]:
dt['agent_dets'] = dt['agent_dets'].apply(lambda x :' '.join([x for x in str(x).split(',') if x not in ['\n',' ']]))

In [8]:
dt['agent_dets'] = dt['agent_dets'].apply(lambda x : [x for x in str(x).split('\xa0') if x not in ['','\n',' ','\n ','  ','\n  ','\n    ','\nO  ','  \n  ','\n \n  ']])

In [9]:
dt['agent_dets_counts'] = dt['agent_dets'].apply(lambda x : len(x))
dt[['agent_registration_1','agent_registration_2','agent_registration_3','agent_registration_4']] = None

In [10]:
for index,row in dt.iterrows():
    if row.agent_dets_counts == 1:
        dt.loc[index,'agent_registration_1'] = row.agent_dets[0]
    elif row.agent_dets_counts == 2:
        if str(row.agent_dets[0])[1] == 'n': 
            dt.loc[index,'agent_registration_2'] = row.agent_dets[0]
            dt.loc[index,'agent_registration_4'] = row.agent_dets[1]
        else:
            dt.loc[index,'agent_registration_1'] = row.agent_dets[0]
            dt.loc[index,'agent_registration_4'] = row.agent_dets[1]
    elif row.agent_dets_counts == 3:
        dt.loc[index,'agent_registration_1'] = row.agent_dets[0]
        dt.loc[index,'agent_registration_2'] = row.agent_dets[1]
        dt.loc[index,'agent_registration_4'] = row.agent_dets[2]
    elif row.agent_dets_counts == 4:
        dt.loc[index,'agent_registration_1'] = row.agent_dets[0]
        dt.loc[index,'agent_registration_2'] = row.agent_dets[1]
        dt.loc[index,'agent_registration_3'] = row.agent_dets[2]
        dt.loc[index,'agent_registration_4'] = row.agent_dets[3]

#### Agent Website

In [11]:
dt['agent_website'] = dt['agent_website'].apply(lambda x :[x for x in str(x).split(',') if x not in ['#']])

In [12]:
dt[['agent_website_1','agent_website_2']] = None

In [13]:
dt[['agent_website_1','agent_website_2']] = dt.apply((lambda x : pd.Series([x.agent_website[0],x.agent_website[-1]])),axis=1)

#### Images

In [14]:
dt['images'] = dt['images'].apply(lambda x :[x for x in str(x).split(',')])

#### Description

In [15]:
dt['description'] = dt.apply((lambda x : str(x.description_1).replace('\n','')),axis = 1)

In [16]:
dt.drop('description_1',axis = 1,inplace =True)

#### Title

In [17]:
dt['title'] = dt.apply((lambda x : x.title[0]),axis = 1)

#### Address

In [18]:
dt['address'] = dt.apply((lambda x : str(x.address).replace('\xa0','').strip()),axis = 1)

#### Added on

In [19]:
dt['added_on'] = dt['added_on'].str.replace(' on','')

In [20]:
dt['added_on'] = dt['added_on'].str.split(',')

In [21]:
dt['added_on_count'] = dt['added_on'].apply(lambda x: len(x))

In [22]:
def handle_added_on(value):
    output = {}
    for x in value:
        if x[0:3].lower() == 'add':
            output['Added'] = str(x).replace('Added','').strip()
        elif x[0:3].lower() == 'upd':
            output['Updated'] = str(x).replace('Updated','').strip()
    if 'Added' in output.keys() and 'Updated' not in output.keys():
        output['Updated'] = None
    elif 'Updated' in output.keys() and 'Added' not in output.keys():
        output['Added'] = None
    return output

In [23]:
dt['added_on_1'] = dt['added_on'].apply(lambda x: handle_added_on(x))

In [24]:
dt = pd.concat([dt,dt['added_on_1'].apply(pd.Series)],axis=1)

In [25]:
dt[dt['added_on_count'] == 2]

,agent_name,agent_dets,agent_website,images,table,title,front_image,details_link,content_title,address,...,agent_registration_1,agent_registration_2,agent_registration_3,agent_registration_4,agent_website_1,agent_website_2,added_on_count,added_on_1,Added,Updated


#### Price

In [26]:
dt['currency'] = dt['price'].apply(lambda x : x[0])

In [27]:
dt['price'] = dt['price'].apply(lambda x: int(''.join(''.join(x[1:]).split(','))))

In [28]:
dt.columns

Index(['agent_name', 'agent_dets', 'agent_website', 'images', 'table', 'title',
       'front_image', 'details_link', 'content_title', 'address',
       'description', 'added_on', 'price', 'agent', 'agent_no', 'features',
       'agent_dets_counts', 'agent_registration_1', 'agent_registration_2',
       'agent_registration_3', 'agent_registration_4', 'agent_website_1',
       'agent_website_2', 'added_on_count', 'added_on_1', 'Added', 'Updated',
       'currency'],
      dtype='object')

#### Agent

In [29]:
dt.agent = dt.agent.str.replace('\n','')

In [30]:
dt.agent_no = dt.agent_no.apply(lambda x: ''.join(x).replace('\n','').replace('\xa0','').strip())

#### Features

In [31]:
###check for even numbers of cols with mod

In [32]:
dt['features_format'] = dt.features.apply(lambda x: [y for y in x if y not in ['sqm']])

In [33]:
dt['features_count'] = dt.features_format.apply(lambda x: len(x))

In [34]:
dt[dt['features_count']%2 != 0]['features_format']

Series([], Name: features_format, dtype: object)

In [35]:
def group_in_twos(value):
    y = {}
    for x in range(0,len(value),2):
        if str(value[x+1])[-1] ==  's':
            key = value[x+1][0:-1]
        else:
            key = value[x+1]
        y[key] = value[x]
    return y

In [36]:
dt['features_format'] = dt.features_format.apply(lambda x: group_in_twos(x))

In [37]:
dt['features_format']

0                                  {'Covered Area': '500'}
1        {'Bedroom': '5', 'Bathroom': '5', 'Toilet': '6...
2        {'Bedroom': '5', 'Bathroom': '5', 'Toilet': '6...
3        {'Bedroom': '5', 'Bathroom': '5', 'Toilet': '6...
4        {'Bedroom': '5', 'Bathroom': '5', 'Toilet': '6...
                               ...                        
26125     {'Bedroom': '4', 'Bathroom': '4', 'Toilet': '5'}
26126     {'Bedroom': '3', 'Bathroom': '4', 'Toilet': '4'}
26127    {'Bedroom': '4', 'Bathroom': '4', 'Toilet': '5...
26128    {'Bedroom': '4', 'Bathroom': '4', 'Toilet': '5...
26129    {'Parking Space': '500', 'Total Area': '500', ...
Name: features_format, Length: 26130, dtype: object

In [38]:
dt = pd.concat([dt,dt['features_format'].apply(pd.Series)],axis=1)

#### Total Area

In [39]:
# dt[~dt['Total Area'].isnull()]

In [40]:
dt['Total Area'] = dt['Total Area'].apply(lambda x: float(str(x).replace(',','')))

#### Covered Area

In [41]:
# dt[~dt['Covered Area'].isnull()]

In [42]:
dt['Covered Area'] = dt['Covered Area'].apply(lambda x: float(str(x).replace(',','')))

#### Table

In [43]:
dt['table_stripped'] = dt['table'].apply(lambda x: str(x).replace('<td width="33.33%">','').replace('<strong>','').replace('</strong>','').replace('</td>','').replace('<td>','').replace('\xa0','').strip())

In [44]:
# dt['table_stripped'][0]

In [45]:
import re

def find_comma_sep_figures(string):
    regx = '([0-9]+(,[0-9]{3})+)'
    return [x[0] for x in re.findall(regx,string)]

In [46]:
def replace_comma_sep_figures(string,lst):
    lst.sort(key=len,reverse = True)
    for x in lst:
        y = x.replace(',','')
        string = string.replace(x,y)
    return string

In [47]:
# xxx=['amy', 'rohan', 'sapna', 'aakash', 'raunak', 'chinmoy', 'muhammad']
# xxx.sort(key=len,reverse = True)
# xxx

In [48]:
dt['table_cleaned'] = dt['table_stripped'].apply(lambda x: replace_comma_sep_figures(str(x),find_comma_sep_figures(str(x))))

# dt['table_identified_numeric_string'] = dt['table_stripped'].apply(lambda x: find_comma_sep_figures(x))
# dt['table_cleaned'] = dt.apply((lambda x: replace_comma_sep_figures(x.table_stripped,x.table_identified_numeric_string)),axis = 1)

In [49]:
# dt['table_identified_numeric_string'] = dt['table_stripped'].apply(lambda x: find_comma_sep_figures(x))
# dt['table_identified_numeric_string_count'] = dt['table_identified_numeric_string'].apply(lambda x: len(x))
# dt[dt['table_identified_numeric_string_count'] > 0]['table_stripped']

In [50]:
def convert_str_to_dict(string):
    dct = {}
    try:
        for x in string.split(','):
            if len(x) > 1:
                y = x.split(':')
            dct[y[0]] = y[1].strip()
        return dct
    except:
        return None

In [51]:
# for x in dt['table_cleaned'][0].split(','):
#     print(x)
# convert_str_to_dict(dt['table_cleaned'][0])
# # 'Property Ref: 1296211'.split(':')[1]

In [52]:
dt['table_dict'] = dt['table_cleaned'].apply(lambda x: convert_str_to_dict(x))

In [53]:
dt[dt['table_dict'].isnull()]['table_stripped']

21730    nan
Name: table_stripped, dtype: object

In [54]:
dt = pd.concat([dt,dt['table_dict'].apply(pd.Series)],axis=1)

In [55]:
# strr = dt[dt['table_dict'].isnull()]['table_cleaned'][20894    ]
# # replace_comma_sep_figures(strr,find_comma_sep_figures(strr))
# convert_str_to_dict(replace_comma_sep_figures(strr,find_comma_sep_figures(strr)))

In [56]:
dt.columns

Index(['agent_name', 'agent_dets', 'agent_website', 'images', 'table', 'title',
       'front_image', 'details_link', 'content_title', 'address',
       'description', 'added_on', 'price', 'agent', 'agent_no', 'features',
       'agent_dets_counts', 'agent_registration_1', 'agent_registration_2',
       'agent_registration_3', 'agent_registration_4', 'agent_website_1',
       'agent_website_2', 'added_on_count', 'added_on_1', 'Added', 'Updated',
       'currency', 'features_format', 'features_count', 'Covered Area',
       'Bedroom', 'Bathroom', 'Toilet', 'Parking Space', 'Total Area',
       'table_stripped', 'table_cleaned', 'table_dict', 'Property Ref',
       'Added On', 'Last Updated', 'Market Status', 'Type', 'Covered Area',
       'Bedrooms', 'Bathrooms', 'Toilets', 'Parking Spaces', 'Total Area',
       'Servicing', 'Service Charge', 'Furnishing'],
      dtype='object')

In [57]:
redundant_columns = ['agent_dets','agent_website','table','added_on','added_on_count','added_on_1','features','agent_dets_counts',
                    'features_format','features_count','table_stripped','table_cleaned', 'table_dict']

In [58]:
dt.drop(redundant_columns,axis = 1,inplace = True)

In [59]:
dt['Service Charge'].unique()

array([nan, '₦500 per annum', '₦356000 per annum', '₦840000 per annum',
       '₦230000 per annum', '₦240000 per annum', '₦500000 per annum',
       '₦250000 per annum', '₦350000 per annum', '₦1200000',
       '₦40000 per month', '₦200000 per annum', '₦1500000 per annum',
       '₦45000 per month', '₦6000000', '₦3100000', '₦1200000 per annum',
       '₦4000000', '₦480000 per annum', '₦76000 per annum',
       '₦50000 per month', '₦60590 per month', '₦3000000',
       '₦800 per annum', '₦1300000 per annum', '₦700000 per month',
       '₦3000000 per annum', '₦700000 per annum', '₦3500000',
       '₦750000 per annum', '₦300000 per annum', '₦1500000',
       '₦400000 per annum', '₦12000', '₦540000 per annum', '₦100000',
       '₦600000 per annum', '₦850000 per annum', '₦296000 per annum',
       '₦296000', '₦45500 per month', '₦800000 per annum',
       '₦30000 per month', '₦450000000 per annum', '₦280000 per annum',
       '₦6666 per annum', '₦2500000 per annum', '₦450000 per annum',
    

In [60]:
dt['Servicing'].unique()

array([nan, 'Serviced'], dtype=object)

In [62]:
# dt['Sharing'].unique()

In [63]:
dt['Covered Area']

,Covered Area,Covered Area
0,500.0,500 sqm
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
26125,NaN,NaN
26126,NaN,NaN
26127,NaN,NaN
26128,NaN,NaN


In [64]:
dt.to_csv('Nigerian_property_sale.csv',index=None)